In [5]:
import os
import lief
import sys

sys.stderr = open(os.devnull, 'w')

In [6]:
def gets_binary_strings(binary):
    strings = []
    # Load the read-only data section
    section = binary.get_section(".rodata")
    if section:
        # Convert the section to a sequence of bytes
        content_bytes = bytes(section.content)
        # Convert the bytes to a string, separated by C-string null-terminator
        strings.extend(content_bytes.decode(errors='ignore').split('\0'))

    return strings

In [9]:
def find_bins_with_keys(fs_root, search_strings):

    # Recursively walk the filesystem
    for root, _, files in os.walk(fs_root):
        # For each file in current directory...
        for file_name in files:

            file_path = os.path.join(root, file_name)

            # Parse the binary
            binary = lief.parse(file_path)

            # Verify that it is a binary
            if binary and isinstance(binary, lief.ELF.Binary):
                # Get the strings from the binary
                binary_strings = gets_binary_strings(binary)

                # Intersect binary strings vs search strings
                match_strs = set(binary_strings) & set(search_strings)
                
                if len(match_strs) == 0:
                    continue

                print(f"Key(s) {match_strs} found in:\n\t{file_path}")

In [ ]:
FS_ROOT_DIR = r'C:\\Users\\dfrwseu2024\\Desktop\\EX6120_EXTRACTED_FILESYSTEM'
find_bins_with_keys(FS_ROOT_DIR, ["ap_mode_cur", "ap_support_mode"])